### AUTHOR: Denvir Higgins
### Last verified: June 2/2023
### Products and set up
### Contact author: Denvir Higgins and Louise Edwards
### Targeted learning level: beginner

The purpose of the notebook is to use the Truth Summary catalog from DP0 DC2 to query nearby, bright galaxy clusters. 

The notebook utilizes aspects from tutorial notebooks 1-8 and is described in three parts below. Within the notebook, you'll see how to porgram a simple equation, as well as make color images, and overplot a circle on an image.

### Part I: Identifying potential BCGs

First, we use the TAP Query to identify large, bright, red galaxies. Then, we create a histogram that bins the redshift of these galaxies, as well as a histogram that bins the apparent magntiude of these galaxies. Then, we create a scatter plot of redshifts versus apparent magnitude. Finally, we convert redshift into distance in parsecs and apparent magnitude into absolute magnitude and plot M vs d.

### Part II: Tracing the Einstein Radius around the BCG

In this section, we investigate the use of the Butler to create a color image of our galaxy, using the right ascension and declination found in our TAP Query from above. Then, we use the distance and estimated total mass of the galaxy to determine the Einstein radius. 

# Part I: Identifying potential BCGs
## A: Setting up the notebook and querying the data

#### We start by setting up imports and plotting parameters.

In [ ]:
# Import general python packages
import numpy as np
import re
import pandas
from pandas.testing import assert_frame_equal
import matplotlib.pyplot as plt

# Import the Rubin TAP service utilities
from lsst.rsp import get_tap_service, retrieve_query

# Astropy
from astropy import units as u
from astropy.coordinates import SkyCoord

# Set the maximum number of rows to display from pandas
pandas.set_option('display.max_rows', 20)

In [ ]:
# Ignore warnings
import warnings
from astropy.units import UnitsWarning
warnings.simplefilter("ignore", category=UnitsWarning)

In [ ]:
# Set up some plotting defaults:
params = {'axes.labelsize': 28,
          'font.size': 24,
          'legend.fontsize': 14,
          'xtick.major.width': 3,
          'xtick.minor.width': 2,
          'xtick.major.size': 12,
          'xtick.minor.size': 6,
          'xtick.direction': 'in',
          'xtick.top': True,
          'lines.linewidth': 3,
          'axes.linewidth': 3,
          'axes.labelweight': 3,
          'axes.titleweight': 3,
          'ytick.major.width': 3,
          'ytick.minor.width': 2,
          'ytick.major.size': 12,
          'ytick.minor.size': 6,
          'ytick.direction': 'in',
          'ytick.right': True,
          'figure.figsize': [8, 8],
          'figure.facecolor': 'White'
          }

plt.rcParams.update(params)

#### Getting ready to use the TAP to search for galaxies. Below we start by showing the available tables, we use "dp02_dc2_catalogs" in this project.

In [ ]:
# Get an instance of the TAP service
service = get_tap_service("tap")
assert service is not None
assert service.baseurl == "https://data.lsst.cloud/api/tap"

In [ ]:
# Query to find out what schemas are in the Rubin TAP_SCHEMA
query = "SELECT * FROM tap_schema.schemas"

# Execute the query
results = service.search(query)

# A TAP Results object is returned
print(type(results))

In [ ]:
# Convert the results to an astropy table and display
results = service.search(query).to_table()
results

#### **Here we set up and execute our TAP Query.** We first use SELECT to highlight the columns of data we want to manipulate, then we use FROM to indicate which catalog we are pulling the data from. In this project, we are querying bright, nearby galaxies, which we indiciate in the TAP Query by using WHERE and AND statements. Finally, we use ORDER to organize the data for analysis. 

#### In this project, we are looking for potential brightest cluster galaxies in the nearby universe. So we choose only bright (apparent magnitude m=11-17) and nearby (redshift z=0.01-0.1). We choose our cuts by analyzing a graph of magnitude versus redshift based on the full sample.

In [ ]:
# Setting up the query for galaxies at z~0.1, apparent magnitude between 11 and 17, and ordering these galaxies 
# by their flux to get the brightest ones at the top of the table
query = "SELECT id, ra, dec, mag_r, redshift, flux_g, flux_r, flux_i, flux_y, flux_z," \
        " truth_type, is_variable "\
        " FROM dp02_dc2_catalogs.TruthSummary " \
        "WHERE mag_r < 17 " \
        "AND mag_r > 11 " \
        "AND redshift <= 0.1 " \
        "AND redshift >= 0.01 " \
        "AND truth_type = 1 " \
        "AND is_variable = 0 " \
        "ORDER by flux_r DESC"
print(query)

In [ ]:
# Execute the query
results = service.search(query)

#### Once we have our results table (~2 minutes to generate), we can use pandas to manipulate the data for analysis. We begin by converting our data into a pandas dataframe.

In [ ]:
galaxies = service.search(query).to_table().to_pandas()

#### We've extracted the ra, dec, redshift, and flux_r of various galaxies. The head(#) function allows us to specify the amount of rows of data we want to access.

In [ ]:
top3 = galaxies.head(3)
top3

## B: Analyzing our galaxies
#### ~40,000 galaxies were returned from the DP0 truth tables. To analyze this data, we create a few histograms to see the distrbutions. First, we create a histogram of redshifts. We queried galaxies with a redshift between 0.01 and 0.1. Try imagining the spread of the data before running the next cell.

#### Using the matplotlib package, we use the command plt.figure() to generate a figure for our histogram.

In [ ]:
fig=plt.figure()
plt.hist(galaxies["redshift"], bins=50)
plt.xlabel("Redshift")
plt.ylabel("Count")
#plt.title("Redshift of 40465 Galaxies at >20 Apparent Magnitude");
plt.savefig('redshift-count.png',format='png', bbox_inches='tight', transparent=True)

#### We will now run the same procedure, this time to see what the spread of the apparent magnitudes (brightnesses) of our galaxies looks like.

In [ ]:
fig=plt.figure()
plt.hist(galaxies["mag_r"], bins=50)
plt.xlabel("Apparent Magnitude")
plt.ylabel("Count")

#plt.title("Apparent Magnitude of 40465 Galaxies at <0.1 Redshift");
plt.savefig('appmag-count.png',format='png', bbox_inches='tight', transparent=True)

#### For further analysis, we can create a scatter plot using another matplotlib package, scatter().

In [ ]:
fig, ax = plt.subplots()
plt.scatter(galaxies["redshift"], galaxies["mag_r"], marker='o')
plt.xlabel("Redshift")
plt.ylabel("Apparent Magnitude")
ax.invert_yaxis()

#plt.title("40465 Galaxies at <0.1 Redshift and >20 Apparent Magnitude");
fig.savefig('appmagvredshift-scatter.png',format='png', bbox_inches='tight', transparent=True)

#### The apparent magnitude of the galaxies can be converted to absolute magnitude using the galaxies' redshifts. The use of the pandas dataframe makes this manipulation fairly simple with the formula below.

In [ ]:
distance=(3e5*1e6*galaxies["redshift"])/69.8 # pc
distance

In [ ]:
absmag=galaxies["mag_r"]-5*np.log10(distance/10.0)

In [ ]:
fig, ax = plt.subplots()
plt.scatter(distance/1e6, absmag, marker='o')
plt.xlabel("Distance [in Mpc]")
plt.ylabel("Absolute Magnitude")
ax.invert_yaxis()
#plt.title("Absolute Magnitude in terms of Distance for 40465 Galaxies");
fig.savefig('absmagvredshift-scatter.png',format='png', bbox_inches='tight', transparent=True)

# Part II: Tracing the Einstein Radius around the potential brightest cluster galaxy (BCG)

#### We start by extracting relevant variables from our TAP Query pandas table to construct an image of the galaxy. The right ascension and declination let the Butler service know where to center the image. The redshift and flux in the r-band are needed to determine the angle size of the Einstein radius, which will be shown after we construct the image. Some discussion about the Einsgtein radius can be found on this web page.
https://lweb.cfa.harvard.edu/~dfabricant/huchra/ay202/lectures/lecture12.pdfhttps://lweb.cfa.harvard.edu/~dfabricant/huchra/ay202/lectures/lecture12.pdf

In [ ]:
# Variables (CHANGE FOR EACH GALAXY)
ra, dec = 58.052836,-34.460247
redshift=0.017737
rflux=66247800.0 #nJy
gflux=41641400.0 #nJy
magr=11.8471

## A: Creating a color image
#### We begin by importing Butler and configuring it to the DP0.2 dataset.

In [ ]:
import lsst
from IPython.display import Markdown as md   
from rubin_jupyter_utils.lab.notebook import get_tap_service, retrieve_query    
#to get the catalog service
service = lsst.rsp.get_tap_service("tap")
#to get the image service
import lsst.daf.butler as dafButler
butler = dafButler.Butler('dp02', collections='2.2i/runs/DP0.2')

In [ ]:
#Run this after making an image, to help from running out of space.
import warnings                      # imports the warnings library
import gc                            # imports python's garbage collector
#remove large amounts of data being stored.
def remove_figure(fig):
    """Remove a figure to reduce memory footprint. """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()      # clear the figure
    plt.close(fig) # close the figure
    gc.collect()   # call the garbage collector

#### Now, we begin setting parameters for our image, which superimposes data from the r, g, and b bands to create a color image. 

In [ ]:
import lsst.geom as geom
def cutout_coadd(butler, ra, dec, band='r', datasetType='deepCoadd',
                 skymap=None, cutoutSideLength=51, **kwargs):
    """
    Produce a cutout from a coadd at the given ra, dec position.

    Adapted from DC2 tutorial notebook by Michael Wood-Vasey.

    Parameters
    ----------
    butler: lsst.daf.persistence.Butler
        Servant providing access to a data repository
    ra: float
        Right ascension of the center of the cutout, in degrees
    dec: float
        Declination of the center of the cutout, in degrees
    band: string
        Filter of the image to load
    datasetType: string ['deepCoadd']
        Which type of coadd to load.  Doesn't support 'calexp'
    skymap: lsst.afw.skyMap.SkyMap [optional]
        Pass in to avoid the Butler read.  Useful if you have lots of them.
    cutoutSideLength: float [optional]
        Size of the cutout region in pixels.

    Returns
    -------
    MaskedImage
    """
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)

    if skymap is None:
        skymap = butler.get("skyMap")

    # Look up the tract, patch for the RA, Dec
    tractInfo = skymap.findTract(radec)
    patchInfo = tractInfo.findPatch(radec)
    xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    patch = tractInfo.getSequentialPatchIndex(patchInfo)

    coaddId = {'tract': tractInfo.getId(), 'patch': patch, 'band': band}
    parameters = {'bbox': bbox}

    cutout_image = butler.get(datasetType, parameters=parameters,
                               dataId=coaddId)

    return cutout_image

#### Depending on the image size, the "cutoutSideLength" parameter may need to be scaled up or down. Generally, 1001 is a good place to start.

In [ ]:
# Select a position at roughly the center of the galaxy cluster:
cutout_image = cutout_coadd(butler, ra, dec, datasetType='deepCoadd',
                            cutoutSideLength=1001)
print("The size of the cutout in pixels is: ", cutout_image.image.array.shape)

In [ ]:
# Make a color figure
from astropy.visualization import make_lupton_rgb

#### The following procedure is pulled from tutorial 6a https://github.com/rubin-dp0/tutorial-notebooks/blob/main/06a_Interactive_Image_Visualization.ipynbhttps://github.com/rubin-dp0/tutorial-notebooks/blob/main/06a_Interactive_Image_Visualization.ipynb

In [ ]:
def createRGB(image, bgr="gri", stretch=1, Q=10, scale=None):
    """
    Create an RGB color composite image.

    Parameters
    ----------
    image : `MultibandExposure`
        `MultibandExposure` to display.
    bgr : sequence
        A 3-element sequence of filter names (i.e., keys of the exps dict)
        indicating what band to use for each channel. If `image` only has
        three filters then this parameter is ignored and the filters
        in the image are used.
    stretch: int
        The linear stretch of the image.
    Q: int
        The Asinh softening parameter.
    scale: list of 3 floats, each less than 1. (default: None)
        Re-scales the RGB channels.

    Returns
    -------
    rgb: ndarray
        RGB (integer, 8-bits per channel) colour image as an NxNx3 numpy array.
    """

    # If the image only has 3 bands, reverse the order of the bands
    #   to produce the RGB image
    if len(image) == 3:
        bgr = image.filters

    # Extract the primary image component of each Exposure with the
    #   .image property, and use .array to get a NumPy array view.

    if scale is None:
        r_im = image[bgr[2]].array  # numpy array for the r channel
        g_im = image[bgr[1]].array  # numpy array for the g channel
        b_im = image[bgr[0]].array  # numpy array for the b channel
    else:
        # manually re-scaling the images here
        r_im = image[bgr[2]].array * scale[0]
        g_im = image[bgr[1]].array * scale[1]
        b_im = image[bgr[0]].array * scale[2]

    rgb = make_lupton_rgb(image_r=r_im,
                          image_g=g_im,
                          image_b=b_im,
                          stretch=stretch, Q=Q)
    # "stretch" and "Q" are parameters to stretch and scale the pixel values

    return rgb

In [ ]:
from lsst.afw.image import MultibandExposure

cutout_image_g = cutout_coadd(butler, ra, dec, band='g',
                              datasetType='deepCoadd', cutoutSideLength=1001)
cutout_image_r = cutout_coadd(butler, ra, dec, band='r',
                              datasetType='deepCoadd', cutoutSideLength=1001)
cutout_image_i = cutout_coadd(butler, ra, dec, band='i',
                              datasetType='deepCoadd', cutoutSideLength=1001)

# Multiband exposures need a list of images and filters
coadds = [cutout_image_g, cutout_image_r, cutout_image_i]
coadds = MultibandExposure.fromExposures(['g', 'r', 'i'], coadds)

In [ ]:
#make a figure with the afw image
fig=plt.figure()
rgb_scaled = createRGB(coadds.image, bgr=['g', 'r', 'i'],
                       scale=[0.6, 0.7, 1.0])
plt.imshow(rgb_scaled, origin='lower')
#plt.savefig('lens1.png')

## B: Determining the Einstein radius
#### Now that we've created our image, we can begin calculating the angular size of the Einstein radius for our galaxy. First, we need to import some constants. 

In [ ]:
# Constants
G=6.6743e-11; #m3 kg-1 s-2
c=299792458; #m/s
ckm=2.99792458e5; #km/s
Msun=1.989e30; # kg
Lsun=3.846e26; #watts

#### One piece of information we need to determine this radius is the average distance from the source to the lensed galaxy. From this paper's Figure 3 (https://academic.oup.com/mnras/article/343/2/639/1042160) I determined an average redshift that I called "ofekavg". We then use this redshift to determine Dls, the distance from the lens to the source. 

In [ ]:
from itertools import pairwise
ofekZ=[0.94,0.69,1.56,0.93,1.34,0.41,1.38,0.83,1.65,0.31,1.24,0.25,1.41,0.36,1.53,0.60,1.72,0.31,1.02,0.44,1.85,0.72,1.17,0.11,1.59,0.41,1.39,0.63,1.52,0.68,1.74,0.25,2.03,0.50]
delZ=[x-y for (x, y) in pairwise(ofekZ)]
ofekavg=sum(delZ[::2])/len(delZ[::2])
ofekavg

In [ ]:
#D=DLS/DS*DL where Dls=distance from lens to source, 
#Dl= distance to source, Ds=distance to lensed galaxy
Dl=(ckm*redshift)/69.8; #Mpc
Dls=(ckm*ofekavg)/69.8
Ds=Dl+Dls
Dls

In [ ]:
# convert distances to meters
Dl_m=Dl*3.086e22 #m
Ds_m=Ds*3.086e22
Dls_m=Dls*3.086e22

#### Now that we've determined our distances, we need to estimate the mass of the galaxy that we just imaged. Using the flux in the r-band and the g-band from the truth table as well as the distance to the source from the location of observation (Earth), we first determine the galaxy's stellar mass using the Erik Bell technique (https://arxiv.org/abs/astro-ph/0302543). 

In [ ]:
# Erik Bell technique using G-R log10(M/L)=a+b*color
#r band = 552 to 691 nm, g band 400 to 552 nm
#converting fluxes to magnitudes
rmag=-2.5*np.log10(rflux)+31.4
gmag=-2.5*np.log10(gflux)+31.4

gr=gmag-rmag

a_g=-0.499
b_g=1.519
a_r=-0.306
b_r=1.097

Lgal=4.0*np.pi*(Dl_m)*(Dl_m)*(rflux*1e-9*1e-26*((c/552e-9)-(c/691e-9))) # Watts

MLrat=10**((a_r)+(b_r)*gr)
Mgal=MLrat*Lgal*(Msun/Lsun)
Mgalsol=Mgal/Msun
print("{:.3e}".format(Mgalsol))

#### We can relate the stellar mass to the halo mass using a power-law rule based on redshift. These constants are for redshift 0<z<0.2 (https://arxiv.org/pdf/2001.02230.pdf)

In [ ]:
#relate stellar mass to cluster mass
#using constants for z=0.0-0.02
A=0.0465#-0.0015
M_A=10**11.77#-0.03
beta=1.0#-0.05
gamma=0.702#-0.006

Mhalo=np.linspace(1e11,1e16,100000)
#Mhalo=[1e10,1e11,1e12,1e13,1e14,1e15]
Mhalo_A=Mhalo/M_A

Mstar=Mhalo*(2*A*(Mhalo_A**-beta+Mhalo_A**gamma)**-1)



In [ ]:
logx=np.log10(Mhalo)
logy=np.log10(Mstar)

fig, ax = plt.subplots()
plt.plot(logx,logy,'r')
plt.ylabel("log(Mstar/Msun)")
plt.xlabel("log(Mhalo/Msun)")
ax.invert_xaxis()
plt.show()
fig.savefig('halo-stellar-mass.png',format='png', bbox_inches='tight')

#### From our distances and the galaxy's halo mass, we compute the Einstein radius in arcseconds from the observed angular value.

In [ ]:
Mhalo=2.308e12*Msun
distance=Dls_m/(Ds_m*Dl_m) 

thetE=np.sqrt(((4*G*Mhalo/(c*c))*distance)) # radians 
thetEdeg=thetE*(180/np.pi) # degrees
thetEsec=thetEdeg*3600 # arcseconds
thetEsec

#### We convert our distance in arcseconds to pixels so that we can superimpose our circle onto our image.

In [ ]:
# 1 pixel is 0.2 arcsecs across
# typical radius range is 3 to 30 ''
radEpx=thetEsec/0.2 #pixels
radEpx

#### Finally, the plot we've been waiting for...

In [ ]:
circle = plt.Circle((499, 499), radEpx, color='g', fill=False)
fig=plt.figure()

ax = plt.gca()
ax.cla() # clear things for fresh plot

ax.imshow(rgb_scaled, origin='lower')
    
ax.add_patch(circle)
fig.savefig('z=0.1-lens1.png')
## lens small, compare w published lens calculations of similar redshifts (NASA/ADS 'einstein radius nearby universe')